In [67]:
import ast
import copy
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit

DAY = 5
YEAR = 2023

current_day is only available in December (EST)


In [64]:
# use test data
raw_test = """input"""

# use real data
raw = get_data(day=DAY, year=YEAR)

In [65]:
def parse_data(data):
    data = data.split("\n")
    #data = [d.split(" ") for d in data]
    #data = [d.split('\n') for d in data.split('\n\n')]
    #data = [np.array(d.split('\n'), dtype=int) for d in data.split('\n\n')]

    return data


dummy = parse_data(raw_test)
real = parse_data(raw)

real

['seeds: 28965817 302170009 1752849261 48290258 804904201 243492043 2150339939 385349830 1267802202 350474859 2566296746 17565716 3543571814 291402104 447111316 279196488 3227221259 47952959 1828835733 9607836',
 '',
 'seed-to-soil map:',
 '3680121696 1920754815 614845600',
 '1920754815 3846369604 448597692',
 '193356576 570761634 505124585',
 '2369352507 2535600415 31531965',
 '2400884472 2567132380 1279237224',
 '0 459278395 111483239',
 '698481161 97868205 361410190',
 '1059891351 0 15994868',
 '111483239 15994868 81873337',
 '',
 'soil-to-fertilizer map:',
 '1633669237 1273301814 72865265',
 '2398515176 2671190790 99210785',
 '2397916384 3018946373 598792',
 '4034325916 3061716397 20017393',
 '3298612516 3793795301 14249501',
 '4030007411 3051046904 2833129',
 '1906984482 224872691 14620134',
 '864506893 1590633724 149044542',
 '1029530319 442871336 36727018',
 '1921604616 770934113 68546178',
 '3560536321 3114405501 28822192',
 '1019762634 1263534129 9767685',
 '3852235341 3579014

# Part 1

In [83]:

def mapping_function(value, src_start, dst_start, length):
    assert type(value) == int
    assert type(src_start) == int
    assert type(dst_start) == int
    assert type(length) == int
    assert value >= src_start
    
    diff = value - src_start
    if diff >= length:
        mapped_value = value
    else:
        mapped_value = dst_start + diff
    
    return mapped_value

def parse_data(all_data):
    mapping_names = []
    mappings = {}
    # parse data
    for data in all_data:
        if "seeds" in data:
            seeds = [int(d) for d in data.split(" ") if d.isnumeric()]
            continue
        if "map" in data:
            mapping_names.append(data.split(" ")[0])
            mappings[mapping_names[-1]] = {}
        elif data != '':
            [destination_start, source_start, length] = data.split(" ")
            mappings[mapping_names[-1]][int(source_start)] = [int(destination_start), int(length)]
            
    return seeds, mappings, mapping_names

def find_lowest_location(seeds, mappings, mapping_names):
    lowest_location_number = -1

    for seed in seeds:
        val = seed
        for mn in mapping_names:
            sorted_src_starts = sorted(list(mappings[mn].keys()), reverse=True)
            for i, s in enumerate(sorted_src_starts):
                if val >= s:
                    src_start = sorted_src_starts[i]
                    [dst_start, ln] = mappings[mn][src_start]
                    val = mapping_function(val, src_start, dst_start, ln)
                    break
        if val < lowest_location_number or lowest_location_number == -1:
            lowest_location_number = val
    
    return lowest_location_number

seeds, mappings, mapping_names = parse_data(real)
result = find_lowest_location(seeds, mappings, mapping_names)
print(result)

525792406


In [52]:
submit(result, part="a", day=DAY, year=YEAR)

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


<Response [200]>

# Part 2

In [105]:
def pairwise(iterable):
    a = iter(iterable)
    return zip(a, a)

#reverse mapping
def parse_data(all_data, reversed=False):
    mapping_names = []
    mappings = {}
    # parse data
    for data in all_data:
        if "seeds" in data:
            seeds = [int(d) for d in data.split(" ") if d.isnumeric()]
            continue
        if "map" in data:
            if reversed:
                name = "-".join(data.split(" ")[0].split("-")[::-1])
                mapping_names.insert(0, name)
                mappings[mapping_names[0]] = {}
            else:
                name = data.split(" ")[0]
                mapping_names.insert(len(mapping_names), name)
                mappings[mapping_names[-1]] = {}
            
        elif data != '':
            [destination_start, source_start, length] = data.split(" ")
            if reversed:
                mappings[mapping_names[0]][int(destination_start)] = [int(source_start), int(length)]
            else:
                mappings[mapping_names[-1]][int(source_start)] = [int(destination_start), int(length)]
            
    return seeds, mappings, mapping_names

def seed_exists(possible_seed, seed_data):
    for seed_start, seed_range in pairwise(seed_data):
        if possible_seed >= seed_start and possible_seed <= (seed_start+seed_range):
            return True
        
    return False

#map from location to seed
def find_seed(seed_data, mappings, mapping_names):
    highest_location = sorted(list(mappings[mapping_names[0]].keys()))[-1]
    # for loc in range(0, highest_location, 1000): # used this to get the rough location
    for loc in range(79000000, highest_location): # found the exact location with this
        val = loc
        for mn in mapping_names:
            sorted_src_starts = sorted(list(mappings[mn].keys()), reverse=True)
            for i, s in enumerate(sorted_src_starts):
                if val >= s:
                    src_start = sorted_src_starts[i]
                    [dst_start, ln] = mappings[mn][src_start]
                    val = mapping_function(val, src_start, dst_start, ln)
                    break
                
        #check if seed exists
        if seed_exists(val, seed_data):
            print(loc, val)
            return loc
        elif loc%10000000 == 0:
            print(loc, val)
    
    return -1

# check if seed exists for the tested location
seeds, mappings, mapping_names = parse_data(real, reversed=True)
seed_index = find_seed(seeds, mappings, mapping_names)
result = seed_index
print(result)

79004094 2302307275
79004094


In [104]:
submit(result, part="b", day=DAY, year=YEAR)

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 5! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>